In [50]:
from langchain_ollama import OllamaLLM
import langchain
import numpy
import pandas


#initialize model
lafilamod= OllamaLLM(
    model="Llama3:latest",
    temperature=0,
    num_ctx=2048,
    verbose=True,
    
)

**pypdf>> PyPDF2** 

also pdfplumber

In [112]:
from pypdf import PdfReader

# Read pdf, get text
reader = PdfReader('/Users/abhimanyu/Downloads/Hands-On_Large_Language_Models_compressed.pdf')
page_ids = []
page_texts = []
for i, p in enumerate(reader.pages):
    page_ids.append(str(i))
    page_texts.append(p.extract_text())



In [113]:
print(page_texts)

['', 'Praise for Hands-On Large Language Models\nThis is an exceptional guide to the world of language models and their\npractical applications in industry. Its highly-visual coverage of\ngenerative, representational, and retrieval applications of language\nmodels empowers readers to quickly understand, use, and refine LLMs.\nHighly recommended!\n—Nils Reimers, Director of Machine Learning at Cohere |\ncreator of sentence-transformers\nJay and Maarten have continued their tradition of providing beautifully\nillustrated and insightful descriptions of complex topics in their new\nbook. Bolstered with working code, timelines, and references to key\npapers, their book is a valuable resource for anyone looking to\nunderstand the main techniques behind how Large Language Models are\nbuilt.\n—Andrew Ng, founder of DeepLearning.AI\nI can’t think of another book that is more important to read right now. On\nevery single page, I learned something that is critical to success in this\nera of langu

In [114]:
# from langchain.text_splitter import MarkdownTextSplitter

# # Create a MarkdownTextSplitter for semantic chunking
# semantic_splitter = MarkdownTextSplitter(
#     chunk_size=200,    # Number of characters per chunk
#     chunk_overlap=50   # Overlap to preserve context
# )

# # Split the text into semantic chunks
# semantic_chunks = semantic_splitter.split_text(page_texts)
# print("\nSemantic Chunks (Markdown-based):\n", semantic_chunks)


In [117]:
# #now embeddings for the text
# from sentence_transformers import SentenceTransformer

# embed_model=SentenceTransformer("")

# text_embeddings=embed_model.encode(page_texts)

# index_embeddings=embed_model.encode(page_ids)
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
embed_model=HuggingFaceBgeEmbeddings(model_name='thenlper/gte-small')

from langchain.vectorstores import FAISS
metadata= [{"page":int(i)} for i in page_ids]
database= FAISS.from_texts(page_texts, embed_model, metadatas=metadata)


In [118]:
from langchain.prompts import PromptTemplate

#query rewriting
# def querewrite(query):
rewtemp=(
        "You are an AI assistant. Rewrite the following user query into a clear, concise search query suitable for retrieving relevant documents. "
        "User Query:+ {query} \nRewritten Query:"
    )
rewprompt= PromptTemplate(template= rewtemp, input_variables=['query'])



In [119]:
chain1=  rewprompt | lafilamod

In [120]:

document_prompt = PromptTemplate(
    input_variables=["page_content"],
     template="{page_content}"
)

In [121]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import RegexParser
parser= RegexParser(
    regex=r"answer: (.*?)\nScore:(.*)",
    output_keys=["answer","score"],
)
# template1="""
# <|begin_of_text|><|start_header_id|>system<|end_header_id|>
# you are a search retrieval agent that answers precisely to the query.the format needs to be like:
# Answer: <some answer>
# Score: <number between 0 and 1>
# <|eot_id|>
# <|start_header_id|>user<|end_header_id|>
# provide a relevant answer according to the user query:
# {context}
# the format needs to be like:
# Answer: <some answer>
# Score: <number between 0 and 1>

# <|eod_id|>
# <|start_header_id|>assistant<|end_header_id|>"""

from langchain.prompts import PromptTemplate

# rag_prompt = PromptTemplate.from_template("""
# Given the following document and a question, provide a helpful answer and a score between 0 and 1 indicating how well the document answers the question.

# Document:
# {context}

# Question:
# {question}

# Answer the question and rate the relevance.

# Output format:
# Answer: <your answer>
# Score: <score between 0 and 1>
# """)
template1map="""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an intelligent assistant helping to answer a user's question using a provided document.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
provide a relevant answer according to the user query:
{question}.

Document: {summaries}

the format needs to be like:
Answer: <some answer>
Score: <number between 0 and 1>

<|eod_id|>
<|start_header_id|>assistant<|end_header_id|>"""

template1combine="""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a smart assistant combining multiple partial answers from different documents to answer a user’s question.
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
question:{question}
partial answers: {summaries}

based on the above construct a concrete final answer for the user:
<|eod_id|>
<|start_header_id|>assistant<|end_header_id|>"""

map_prompt= PromptTemplate(template=template1map, input_variables=[ "question","summaries"])
combine_prompt= PromptTemplate(template=template1combine, input_variables=["question", "summaries"])
document_variable_name="summaries"
# from langchain.chains import RetrievalQA
# rag= RetrievalQA.from_chain_type(llm=lafilamod, chain_type="map_rerank", retriever= database.as_retriever(), chain_type_kwargs={'prompt': rag_prompt},output_parser=parser, verbose=True,)

In [122]:
from langchain.chains import RetrievalQA, LLMChain, MapReduceDocumentsChain, StuffDocumentsChain, ReduceDocumentsChain

# rag= RetrievalQA.from_chain_type(llm=lafilamod, chain_type="map_reduce", retriever= database.as_retriever(),chain_type_kwargs={'map_prompt': map_prompt,"combine_prompt": combine_prompt})

llm_chain_map= LLMChain(llm=lafilamod, prompt= map_prompt)
llm_chain_combine=LLMChain(llm=lafilamod, prompt=combine_prompt)

combine_docschain=StuffDocumentsChain(llm_chain=llm_chain_combine,     document_prompt=document_prompt,
document_variable_name=document_variable_name)

reduce_doc_chain=ReduceDocumentsChain(combine_documents_chain=combine_docschain)

map_reduce_chain= MapReduceDocumentsChain(
    llm_chain= llm_chain_map,
    reduce_documents_chain= reduce_doc_chain,
    document_variable_name=document_variable_name
)
from langchain.chains import RetrievalQA

rag= RetrievalQA( retriever= database.as_retriever(), combine_documents_chain= map_reduce_chain )

In [123]:

chain2= rag | chain1

In [124]:
rag_response=chain2.invoke("what is an llm agent?")
print(rag_response)


Here's a rewritten version of your query that can help retrieve relevant documents:

**Search Query:** "Large Language Model (LLM) Agent Definition"

This rewritten query uses specific keywords and phrases to help search engines or databases quickly identify relevant documents. The use of parentheses around "Large Language Model" helps to clarify the meaning of LLM, which is a type of AI model.


In [125]:
template1="""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
you are a search retrieval agent that answers precisely to the query. 
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
provide a relevant answer according to the user query:
{question}
Relevant text:
{context}

<|eod_id|>
<|start_header_id|>assistant<|end_header_id|>"""

stuff_prompt=PromptTemplate(template= template1, input_variables=["question","context"])
ragstuff= RetrievalQA.from_chain_type(llm=lafilamod, chain_type="stuff", retriever= database.as_retriever(), chain_type_kwargs={'prompt': stuff_prompt, 'document_variable_name': 'context'}, verbose=True,)

/var/folders/ql/hdbyk3lx6vq0x6fs9227fc_m0000gn/T/ipykernel_50628/3518605215.py:15: ResourceWarning: unclosed <socket.socket fd=83, family=2, type=1, proto=6, laddr=('127.0.0.1', 57614), raddr=('127.0.0.1', 11434)>
  ragstuff= RetrievalQA.from_chain_type(llm=lafilamod, chain_type="stuff", retriever= database.as_retriever(), chain_type_kwargs={'prompt': stuff_prompt, 'document_variable_name': 'context'}, verbose=True,)


In [126]:
def q(question):
    response= ragstuff.invoke(question)
    return response


In [127]:
q("what is an llm agent, how to use it for search retrieval")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'what is an llm agent, how to use it for search retrieval',
 'result': '**What is an LLM Agent?**\n\nAn LLM (Large Language Model) agent is a system that leverages a language model to determine its actions and make decisions. It can interact with the real world through the use of tools, allowing it to perform tasks beyond what an LLM can do in isolation.\n\n**How to Use an LLM Agent for Search Retrieval**\n\nTo use an LLM agent for search retrieval, you can follow these steps:\n\n1. **Create a tool**: Define a tool that the agent can use to interact with the outside world. For example, you can create a tool that allows the agent to access a web search engine like DuckDuckGo.\n2. **Prepare tools**: Load the tools into an array and pass them to the LLM agent.\n3. **Create the ReAct agent**: Use the `create_react_agent` function from the LangChain library to create an instance of the ReAct agent.\n4. **Pass the agent to the AgentExecutor**: Pass the ReAct agent to the `AgentExec

In [128]:
#adding conversation summary to the llm now
##but do we need memory??##

what we need is. a better retrieval logic that very accurately takes on to the right page(end goal) 
but right now we need that **at least the top 2** has the right one in the search

In [129]:
#current retrieval not great but fast
results = database.similarity_search("agentic RAG", k=2)
for doc in results:
    page = doc.metadata.get("page")
    print(f"Page: {page}\nContent: {doc.page_content}\n")

Page: 356
Content: Query routing
An additional enhancement is to give the model the ability to search
multiple data sources. We can, for example, specify for the model that if it
gets a question about HR, it should search the company’s HR information
system (e.g., Notion) but if the question is about customer data, that it
should search the customer relationship management (CRM) (e.g.,
Salesforce).
Agentic RAG
You may be able to now see that the list of previous enhancements slowly
delegates more and more responsibility to the LLM to solve more and more
complex problems. This relies on the LLM’s capability to gauge the
required information needs as well as its ability to utilize multiple data
sources. This new nature of the LLM starts to become closer and closer to
an agent that acts on the world. The data sources can also now be abstracted
into tools. We saw, for example, that we can search Notion, but by the same
token, we should be able to post to Notion as well.
Not all LLMs will h

In [130]:
query={'query':"what is agentic rag"}
actual_query=chain1.invoke(query)


In [132]:
print(actual_query)


Here's a rewritten version of the user query that can help retrieve relevant documents:

**Search Query:** "definition of agentic role" OR "characteristics of agentic behavior"

This rewritten query uses specific keywords and phrases to capture the essence of the original question. By using terms like "definition", "role", and "behavior", we can search for documents that provide a clear explanation of what an "agentic rag" is, or related concepts that describe its characteristics.


In [133]:
#current retrieval not great but fast
results = database.similarity_search("actual_query", k=2)
for doc in results:
    page = doc.metadata.get("page")
    print(f"Page: {page}\nContent: {doc.page_content}\n")

Page: 329
Content: you train a retrieval model on internet and Wikipedia data, and then deploy
it on legal texts (without having enough legal data as part of the training
set), the model will not work as well in that legal domain.
The final thing we’d like to point out is that this is a case where each
sentence contained a piece of information, and we showed queries that
specifically ask for that information. What about questions whose answers
span multiple sentences? This highlights one of the important design
parameters of dense retrieval systems: what is the best way to chunk long
texts? And why do we need to chunk them in the first place?
Chunking long texts
One limitation of Transformer language models is that they are limited in
context sizes, meaning we cannot feed them very long texts that go above
the number of words or tokens that the model supports. So how do we
embed long texts?
There are several possible ways, and two possible approaches shown in
Figure 8-7 include indexin

In [204]:
#implementing keyword search
# from sklearn.feature_extraction import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
import numpy as np
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
from tqdm import tqdm
def tokenizer(text):
    tokenizer_corpus=[]
    for text in text.lower().split():
        text=text.strip(string.punctuation)
        if len(text)>0 and text not in _stop_words.ENGLISH_STOP_WORDS:
            tokenizer_corpus.append(text)
    return tokenizer_corpus



In [231]:
tokenized_doc=[]
for passage in tqdm(page_texts):
    tokenized_doc.append(tokenizer(passage))


bm25= BM25Okapi(tokenized_doc) #notice how naming it tokenize_doc=... will give error later 
def keyword_search(query, top_k=5, num_candidates=10):
    # print("input question:", query)

    bm_25_scores= bm25.get_scores(tokenizer(query))
    bm_25_scores= bm_25_scores / (bm_25_scores.max() + 1e-9) #adding a max normalization layer for values between 0 and 1

    top_n= np.argpartition(bm_25_scores, -num_candidates)[-num_candidates:]
    bm25_hits= [{'corpus_id': idx , 'score':bm_25_scores[idx]} for idx in top_n]
    # bm25_hits= [{ idx ,bm_25_scores[idx]} for idx in top_n]

    bm25_hits=sorted(bm25_hits, key= lambda x: x['score'], reverse=True)

    scores_only=[]

    # print("top 5 results")
    # for hit in bm25_hits[0:top_k]:
    #     print("\t{:.3f}\t".format(hit['score']),
    #     page_texts[hit['corpus_id']].replace("\n"," ")) # fetch the content then replace line breaks with space

    #     scores_only.append(hit['score'])
    # print(scores_only)
    return  [(hit['corpus_id'], hit['score']) for hit in bm25_hits]


  0%|          | 0/598 [00:00<?, ?it/s]

100%|██████████| 598/598 [00:00<00:00, 29981.16it/s]


In [232]:


keyword_search("what is agentic rag")



[(575, 0.999999999921749),
 (561, 0.8969531239724363),
 (536, 0.8716653374604687),
 (356, 0.6711819067485029),
 (542, 0.5235863667105769),
 (562, 0.45513670097404435),
 (348, 0.4118913815791562),
 (354, 0.37692730331059543),
 (318, 0.3381180448580582),
 (353, 0.32935187669117916)]

In [233]:
keyword_scores_with_id=keyword_search("what is agentic Rag?")
keyword_scores_with_id

[(575, 0.999999999921749),
 (561, 0.8969531239724363),
 (536, 0.8716653374604687),
 (356, 0.6711819067485029),
 (542, 0.5235863667105769),
 (562, 0.45513670097404435),
 (348, 0.4118913815791562),
 (354, 0.37692730331059543),
 (318, 0.3381180448580582),
 (353, 0.32935187669117916)]

**langchain stores the text in doc.page_content and metadata in .metadata**

In [226]:
#this is chatgpt code-- not needed

# from rank_bm25 import BM25Okapi
# from sklearn.feature_extraction import _stop_words
# import string
# from tqdm import tqdm
# import numpy as np

# # Tokenizer
# def tokenizer(text):
#     tokens = []
#     for t in text.lower().split():
#         t = t.strip(string.punctuation)
#         if len(t) > 0:  # no stopword filtering unless you want it
#             tokens.append(t)
#     return tokens

# # Tokenize corpus
# tokenized_doc = [tokenizer(p) for p in tqdm(page_texts)]

# # Fit BM25
# bm25 = BM25Okapi(tokenized_doc)

# def keyword_search(query, min_score=0.0):
#     query_tokens = tokenizer(query)
#     scores = bm25.get_scores(query_tokens)
    
#     # Keep all matches above min_score
#     hits = [
#         {"idx": idx, "score": score, "text": page_texts[idx]}
#         for idx, score in enumerate(scores)
#         if score > min_score
#     ]
    
#     # Sort by score (highest first)
#     hits = sorted(hits, key=lambda x: x["score"], reverse=True)
    
#     return hits

# # Example
# results = keyword_search("langchain", min_score=0)
# for hit in results:
#     print(f"Score: {hit['score']:.3f} | {hit['text'][:80]}...")


In [235]:

results = [
    {
    'page-number': doc.metadata.get("page"),
    'score' : score,
    'text': doc.page_content
    }

    for doc, score in database.similarity_search_with_relevance_scores("what is agentic rag", k=5)
]
print(results)
scores= [(item['page-number'],item['score']) for item in results]
print(scores)

[{'page-number': 356, 'score': 0.8307425764322219, 'text': 'Query routing\nAn additional enhancement is to give the model the ability to search\nmultiple data sources. We can, for example, specify for the model that if it\ngets a question about HR, it should search the company’s HR information\nsystem (e.g., Notion) but if the question is about customer data, that it\nshould search the customer relationship management (CRM) (e.g.,\nSalesforce).\nAgentic RAG\nYou may be able to now see that the list of previous enhancements slowly\ndelegates more and more responsibility to the LLM to solve more and more\ncomplex problems. This relies on the LLM’s capability to gauge the\nrequired information needs as well as its ability to utilize multiple data\nsources. This new nature of the LLM starts to become closer and closer to\nan agent that acts on the world. The data sources can also now be abstracted\ninto tools. We saw, for example, that we can search Notion, but by the same\ntoken, we shoul

In [236]:
print(scores)
print(keyword_scores_with_id)

[(356, 0.8307425764322219), (318, 0.8224408695341826), (316, 0.8025796940090302), (562, 0.7958404972258353), (357, 0.7892711100952907)]
[(575, 0.999999999921749), (561, 0.8969531239724363), (536, 0.8716653374604687), (356, 0.6711819067485029), (542, 0.5235863667105769), (562, 0.45513670097404435), (348, 0.4118913815791562), (354, 0.37692730331059543), (318, 0.3381180448580582), (353, 0.32935187669117916)]


both scores and keyword_score_with_id are tuples so we convert em to dicts so they(scores) are mapped to the page_ids

In [240]:
bm25_dict = dict(keyword_scores_with_id)       # {356: 0.8307, 421: 0.7923, ...}
vector_dict = dict(scores)   # same format for vector search
alpha=0.6
combined = []
for doc_id in set(bm25_dict) | set(vector_dict):  # all doc IDs from both searches
    bm25_score = bm25_dict.get(doc_id, 0.0)
    vector_score = vector_dict.get(doc_id, 0.0)
    final_score = alpha * bm25_score + (1 - alpha) * vector_score
    combined.append((doc_id, final_score))

# sort by final score, then fetch text
combined.sort(key=lambda x: x[1], reverse=True)
for doc_id, score in combined[:5]:
    print(score, page_texts[doc_id])


0.7350061746219905 Query routing
An additional enhancement is to give the model the ability to search
multiple data sources. We can, for example, specify for the model that if it
gets a question about HR, it should search the company’s HR information
system (e.g., Notion) but if the question is about customer data, that it
should search the customer relationship management (CRM) (e.g.,
Salesforce).
Agentic RAG
You may be able to now see that the list of previous enhancements slowly
delegates more and more responsibility to the LLM to solve more and more
complex problems. This relies on the LLM’s capability to gauge the
required information needs as well as its ability to utilize multiple data
sources. This new nature of the LLM starts to become closer and closer to
an agent that acts on the world. The data sources can also now be abstracted
into tools. We saw, for example, that we can search Notion, but by the same
token, we should be able to post to Notion as well.
Not all LLMs will h

In [237]:
combined_results=[]
alpha=0.5
for doc_id in page_ids:
    combined_score=alpha*scores[doc_id]+ (1-alpha)*keyword_scores_with_id[doc_id]
    combined_results.append((doc_id, combined_score))


TypeError: list indices must be integers or slices, not str

In [92]:
# from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever
# retriever = WeaviateHybridSearchRetriever(
#     alpha = 0.5,               # defaults to 0.5, which is equal weighting between keyword and semantic search
#     client = query,           # keyword arguments to pass to the Weaviate client
#     index_name = "page_ids",  # The name of the index to use
#     text_key = "page_texts",         # The name of the text key to use
#     attributes = [],           # The attributes to return in the results
# )